# Data Inspection- Preprocessing - Unsupervised ML

This tutorial demonstrates the most important basic steps involved in the analysis of scanning electron diffraction data.

The data was acquired from a GaAs nanowire adopting the zinc blende structure and exhibiting type I twinning (i.e. on {111}) along its length. The region of interest contains a single nanowire comprising multiple crystals each in one of the two twinned orientations and near to a <1-10> zone axis.

This functionaility has been checked to run in pyxem version 0.13.1 (April 2021). However, bugs are always possible, do not trust the code blindly, and if you experience any issues please report them here: https://github.com/pyxem/pyxem-demos/issues

## Contents

1. <a href='#loa'> Loading & Inspection</a>
2. <a href='#cal'> Alignment & Calibration</a>
3. <a href='#vdf'> Virtual Diffraction Imaging</a>
4. <a href='#ml'> Machine Learning SPED Data</a>
5. <a href='#vec'> Peak Finding</a>

Import pyxem and other required libraries

In [ ]:
# Changing the matplotlib background will give you interactive 
#%matplotlib qt5

In [ ]:
%matplotlib inline
import hyperspy.api as hs
import pyxem as pxm
import numpy as np

<a id='loa'></a>

##  1. Loading and Inspection

Load the SPED data acquired from the nanowire

In [ ]:
dp = hs.load('./data/01/twinned_nanowire.hdf5')

Inspect the dp object

In [ ]:
dp

Specify that the data is electron diffraction data

In [ ]:
dp.set_signal_type('electron_diffraction')

Inspect the signal type

In [ ]:
dp

Inspect the data type of the object

In [ ]:
dp.data.dtype

Inspect the metadata associated with the object 'dp'

In [ ]:
dp.metadata

Set important experimental parameters using the built in function

In [ ]:
dp.set_experimental_parameters(beam_energy=300.0,
                               camera_length=21.0,
                               scan_rotation=277.0,
                               convergence_angle=0.7,
                               exposure_time=10.0)

See how this changed the metadata

In [ ]:
dp.metadata

Set another metadata item and check it

In [ ]:
dp.metadata.set_item("General.title", 'GaAs Nanowire')
dp.metadata

In [ ]:
dp

Plot the data to inspect it

In [ ]:
dp.plot(cmap='inferno', vmax=50)

<a id='cal'></a>

## 2. Alignment & Calibration

Apply distortion corrections to the data due to off-axis acquisition

In [ ]:
dp.apply_affine_transformation(np.array([[0.99,0,0],
                                         [0,0.69,0],
                                         [0,0,1]]),
                               keep_dtype=True)

In [ ]:
dp.data.dtype

Align the dataset based on the direct beam position

In [ ]:
dp.center_direct_beam(method='cross_correlate',
                      radius_start=2,
                      radius_finish=5,
                      half_square_width=10)

In [ ]:
dp.plot(cmap='inferno', vmax=50)

Measure known interplanar spacing to obtain calibration

In [ ]:
dpm = dp.mean((0,1))
dpm.plot(cmap='inferno', vmax=50)
line = hs.roi.Line2DROI(x1=25.8525, y1=64.5691, x2=120.907, y2=77.0079, linewidth=5.49734)
line.add_widget(dpm)

In [ ]:
trace = line(dpm)
trace.plot()

In [ ]:
recip_d111 = np.sqrt((3/5.6535**2))
recip_cal = recip_d111 / 11.4

Set data calibrations

In [ ]:
dp.set_diffraction_calibration(recip_cal)
dp.set_scan_calibration(10)

Plot the calibrated data

In [ ]:
dp.plot(cmap='inferno', vmax=50)

<a id='vdf'></a>

##  3. Virtual Diffraction Imaging & Selecting Regions

### 3.1 Interactive VDF Imaging

Plot an interactive virtual image integrating intensity within a circular subset of pixels in the diffraction pattern

In [ ]:
roi = hs.roi.CircleROI(cx=0.,cy=0, r_inner=0, r=0.07)

In [ ]:
dp.plot_integrated_intensity(roi)

Get the virtual diffraction image associated with the last integration window used interactively

In [ ]:
vdf = dp.get_integrated_intensity(roi)

Plot the virtual dark-field image

In [ ]:
vdf.plot()

Inspect the metadata

In [ ]:
vdf.metadata

Save the virtual dark-field image as a 32bit tif

In [ ]:
vdf.change_dtype('float32')
vdf.save('vdfeg.tif')

### 3.2 Form multiple images using a VDF Generator

Import the VirtualImageGenerator class

In [ ]:
from pyxem.generators.virtual_image_generator import VirtualImageGenerator

Initialize the VDFGenerator

In [ ]:
vdfgen = VirtualImageGenerator(dp)

Calculate 10 annular VDF images between 0 and 1 reciprocal angstroms

In [ ]:
vdfs = vdfgen.get_concentric_virtual_images(k_min=0,
                                        k_max=1,
                                        k_steps=10)

Plot the VDF images

In [ ]:
vdfs.plot()

Save the stack of VDF image as a 32bit tif stack

In [ ]:
vdfs.change_dtype('float32')
vdfs.save('vdsfeg.tif')

### 3.3 Select a region in the scan

Plot the data with an adjustable marker indicating where to crop the scan region

In [ ]:
reg = hs.roi.RectangularROI(left=50, top=630, right=290, bottom=870)
dp.plot(cmap='inferno', vmax=50)
reg.add_widget(dp)

Crop the dataset based on the region defined above

In [ ]:
dpc = reg(dp)

Calculate the mean diffraction pattern from the selected region

In [ ]:
dpcm = dpc.mean((0,1))

Plot the mean diffraction pattern from the selected region

In [ ]:
dpcm.plot(cmap='inferno', vmax=50)

<a id='ml'></a>


## 4. Unsupervised learning

Perform singular value decomposition (SVD) of the data

In [ ]:
dpc.data = dpc.data.astype('float64')
dpc.decomposition(True, algorithm='SVD')

Obtain a "Scree plot" by plotting the fraction of variance described by each principal component

In [ ]:
dpc.plot_explained_variance_ratio()

Plot the decomposition results and have a look at them

In [ ]:
dpc.plot_decomposition_results()

Perform non-negative matrix factorisation (NMF)

In [ ]:
dpc.decomposition(True, algorithm='NMF', output_dimension=3)

Plot the NMF results

In [ ]:
dpc.plot_decomposition_results()

<a id='vec'></a>

##  5. Peak Finding

Perform peak finding on all diffraction patterns in data

In [ ]:
peaks = dpc.find_peaks(method='difference_of_gaussian',
                       min_sigma=1.,
                       max_sigma=6.,
                       sigma_ratio=1.6,
                       threshold=0.04,
                       overlap=0.99,
                       interactive=False)

Check the peaks object type

In [ ]:
peaks

Look at what's in the peaks object

In [ ]:
print(peaks.inav[0,0].data)
print(" ")
print(peaks.inav[1,1].data)

coaxing peaks back into a DiffractionVectors

In [ ]:
from pyxem.signals.diffraction_vectors import DiffractionVectors

In [ ]:
peaks = DiffractionVectors.from_peaks(peaks,center=(72,72),calibration=recip_cal)

Plot found peak positions as an overlay on the data

In [ ]:
peaks.plot_diffraction_vectors_on_signal(dpc, cmap='gray', vmax=50)

Form a diffracting pixels map to show where peaks were found and plot it

In [ ]:
crystim = peaks.get_diffracting_pixels_map(binary=False)
crystim.plot(cmap='viridis')